In [1]:
%matplotlib inline
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SearchIO
from pybedtools import BedTool

/home/benjamin/anaconda3/lib/python3.5/site-packages/Bio/SearchIO/__init__.py:211: BiopythonExperimentalWarning: Bio.SearchIO is an experimental submodule which may undergo significant changes prior to its future official release.
  BiopythonExperimentalWarning)


In [2]:
source_dir = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'

In [3]:
out_dir = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/TE_analysis'

In [6]:
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [10]:
p_repet_gff = pd.read_csv(source_dir+'/'+ "Pst_104E_v12_p_ctg.REPET.gff", sep='\t', header = None, skiprows=1)

In [12]:
p_repet_gff[8].head()

0    ID=ms7308_pcontig_193_DTX-incomp-chim_MCL5_Pst...
1    ID=mp7308-1_pcontig_193_DTX-incomp-chim_MCL5_P...
2    ID=ms7309_pcontig_193_DTX-incomp_MCL9_Pst79_RE...
3    ID=mp7309-1_pcontig_193_DTX-incomp_MCL9_Pst79_...
4    ID=ms7310_pcontig_193_DTX-incomp_MCL9_Pst79_RE...
Name: 8, dtype: object

In [14]:
TE_post_analysis_p = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/REPET/Pst79_p/Pst79_p_full_annotate/postanalysis'

In [15]:
TE_post_analysis_p_header = 'TE      length  covg    frags   fullLgthFrags   copies  fullLgthCopies  meanId  sdId    minId   q25Id   medId   q75Id   maxId   meanLgth        sdLgth  minLgth q25Lgth medLgth q75Lgth maxLgth meanLgthPerc    sdLgthPerc      minLgthPerc  q25LgthPerc     medLgthPerc     q75LgthPerc     maxLgthPerc'.split(' ')

In [17]:
TE_post_analysis_p_header = [x for x in TE_post_analysis_p_header if x != '']
TE_post_analysis_p_header

['TE',
 'length',
 'covg',
 'frags',
 'fullLgthFrags',
 'copies',
 'fullLgthCopies',
 'meanId',
 'sdId',
 'minId',
 'q25Id',
 'medId',
 'q75Id',
 'maxId',
 'meanLgth',
 'sdLgth',
 'minLgth',
 'q25Lgth',
 'medLgth',
 'q75Lgth',
 'maxLgth',
 'meanLgthPerc',
 'sdLgthPerc',
 'minLgthPerc',
 'q25LgthPerc',
 'medLgthPerc',
 'q75LgthPerc',
 'maxLgthPerc']

In [21]:
p_repet_summary_df = pd.read_csv(TE_post_analysis_p+'/'+'Pst79p_anno_chr_allTEs_nr_noSSR_join_path.annotStatsPerTE.tab' ,\
                                names = TE_post_analysis_p_header, header=None, sep='\t', skiprows=1 )

In [22]:
p_repet_summary_df.head()

,TE,length,covg,frags,fullLgthFrags,copies,fullLgthCopies,meanId,sdId,minId,...,medLgth,q75Lgth,maxLgth,meanLgthPerc,sdLgthPerc,minLgthPerc,q25LgthPerc,medLgthPerc,q75LgthPerc,maxLgthPerc
0,DHX-comp_MCL7_Pst79_REPET-B-G1637-Map3_reversed,6846,83754,57,5,45,5,87.95,8.91,59.2,...,690.0,2511.0,6846.0,27.19,33.33,0.34,3.30,10.08,36.68,100.00
1,DHX-comp_MCL913_Pst79_REPET-B-R1806-Map7,5263,60250,52,2,43,2,64.67,11.98,55.8,...,1287.0,1943.0,5263.0,26.62,24.40,0.53,1.98,24.45,36.92,100.00
2,DHX-incomp-chim_MCL1088_Pst79_REPET-B-R439-Map4,6573,25628,25,2,24,2,74.05,14.87,56.1,...,141.5,1160.0,6593.0,16.25,30.71,0.38,0.64,2.15,17.65,100.30
3,DHX-incomp-chim_MCL275_Pst79_REPET-B-G1603-Map3,6321,58446,65,4,54,4,71.73,13.08,56.5,...,298.5,1276.0,6324.0,17.12,27.02,0.35,1.57,4.72,20.19,100.05
4,DHX-incomp-chim_MCL66_Pst79_REPET-B-R1526-Map5...,6436,64594,64,5,56,5,75.09,10.98,57.1,...,280.0,1485.0,6440.0,17.93,28.32,0.36,1.32,4.35,23.07,100.06
